This notebook is responsible for visualizing the data from the Github API, which should be provided in a serialized format.

It must be possible to run this notebook using jupyterlite.

If you do not see the newest version of the notebook, you have to "delete" it in your jupyterlite instance, as your local version is overriding the remote version provided by the deployment.

(jupyterlite) We need to manually install dependencies for the front-end, even though they are also defined in the `requirements.txt`.

TODO: Fixed versions to prevent version-drift.

### Troubleshooting:

#### `commits.json` cannot be found when loading

This likely occurred because the backend and frontend of jupyterlite are out of sync. As a workaround, delete the cache/cookies for the page and reload it.
Note that you this will reset all notebooks to their versions saved on Github, so download your notebooks if necessary.
You will also have to re-run the notebooks.

In [ ]:
%pip install plotly matplotlib ipywidgets

In [1]:
import json
import datetime
import numpy as np
import plotly
import ipywidgets

Note: We can use the file upload widget instead of requiring the user to already have uploaded the file to jupyterlite.

In [2]:
# Read the commit data from 'commits.json'
with open('commits.json') as f:
	commits = json.load(f)

In [3]:
# Convert all the commit dates to datetime objects for easier handling
commit_dates = [datetime.datetime.strptime(commit['author']['date'], '%Y-%m-%dT%H:%M:%SZ') for commit in commits]

In [11]:
# Setting up the number of sprints
num_sprints = ipywidgets.IntSlider(min=1, max=10, step=1, description='No. of sprints')
display(num_sprints)

IntSlider(value=1, description='No. of sprints', max=10, min=1)

In [14]:
# TODO: It would be optimal to be able to auto-refresh this cell if the one above is changed (== the number of sprints)
# Use an ipywidget to set a start and end datetime for each sprint
print('Set a name and the start and end dates for each sprint below:')

sprint_names = []
start_dates = []
end_dates = []

for i in range(num_sprints.value):
	sprint_names.append(ipywidgets.Text(value=f'Sprint {i+1}', placeholder='Set the name for this Sprint', description='Sprint Name:'))
	start_dates.append(ipywidgets.DatePicker(description='Start Date'))
	end_dates.append(ipywidgets.DatePicker(description='End Date'))
	display(sprint_names[i], start_dates[i], end_dates[i])


Set a name and the start and end dates for each sprint below:


Text(value='Sprint 1', description='Sprint Name:', placeholder='Set the name for this Sprint')

DatePicker(value=None, description='Start Date', step=1)

DatePicker(value=None, description='End Date', step=1)

Text(value='Sprint 2', description='Sprint Name:', placeholder='Set the name for this Sprint')

DatePicker(value=None, description='Start Date', step=1)

DatePicker(value=None, description='End Date', step=1)

In [35]:
# Plot a heatmap for the number of commits per day and hour of the week
def plot_commit_heatmap(commit_dates, start_date=None, end_date=None):
	# Convert date objects to datetime objects
	if start_date is not None:
		start_date = datetime.datetime.combine(start_date, datetime.datetime.min.time())
	if end_date is not None:
		end_date = datetime.datetime.combine(end_date, datetime.datetime.max.time())
	# Filter the dates to only include those between the start and end dates
	# The dates are already sorted, so we can just filter the beginning and end
	if start_date is not None:
		commit_dates = [date for date in commit_dates if date >= start_date]
	if end_date is not None:
		commit_dates = [date for date in commit_dates if date <= end_date]
	
	# Get the day and hour of each commit
	days = [date.weekday() for date in commit_dates]
	y_vals = [date.hour for date in commit_dates]
	
	# Create a 2D histogram of the day and hour
	heatmap, xedges, yedges = np.histogram2d(days, y_vals, bins=(7, 24))
	
	# Create a plotly figure
	fig = plotly.graph_objs.FigureWidget()
	
	# Add axis labels
	fig.update_layout(
		xaxis=dict(
			title='Hour of the day',
			tickmode='array',
			tickvals=np.arange(0, 24, 2),
			ticktext=np.arange(0, 24, 2)
		),
		yaxis=dict(
			title='Day of the week',
			tickmode='array',
			tickvals=np.arange(0, 7),
			ticktext=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
		)
	)
	
	# Add a heatmap trace
	fig.add_trace(plotly.graph_objs.Heatmap(
		z=heatmap,
		x=xedges,
		y=yedges,
		colorscale='oranges',
		colorbar=dict(
			title='Commits'
		),
		hoverinfo='z'
	))
	
	return fig

In [48]:
# Add a widget to select the sprint to display
sprint_selector = ipywidgets.Dropdown(
	options=[(sprint_names[i].value, i) for i in range(num_sprints.value)],
	description='Sprint:',
	disabled=False,
)
display(sprint_selector)

fig = plot_commit_heatmap(commit_dates, start_dates[sprint_selector.value].value, end_dates[sprint_selector.value].value)
fig.update_layout(title=f'Commit timings during {sprint_names[sprint_selector.value].value} ({start_dates[sprint_selector.value].value} - {end_dates[sprint_selector.value].value})')
display(fig)

def update_sprint(change):
	fig.data[0].z = plot_commit_heatmap(commit_dates, start_dates[change['new']].value, end_dates[change['new']].value).data[0].z
	fig.update_layout(title=f'Commit timings during {sprint_names[change["new"]].value} ({start_dates[change["new"]].value} - {end_dates[change["new"]].value})')

sprint_selector.observe(update_sprint, names='value')


Dropdown(description='Sprint:', options=(('Sprint 1', 0), ('Sprint 2', 1)), value=0)

FigureWidget({
    'data': [{'colorbar': {'title': {'text': 'Commits'}},
              'colorscale': [[0.0, 'rgb(255,245,235)'], [0.125,
                             'rgb(254,230,206)'], [0.25, 'rgb(253,208,162)'],
                             [0.375, 'rgb(253,174,107)'], [0.5, 'rgb(253,141,60)'],
                             [0.625, 'rgb(241,105,19)'], [0.75, 'rgb(217,72,1)'],
                             [0.875, 'rgb(166,54,3)'], [1.0, 'rgb(127,39,4)']],
              'hoverinfo': 'z',
              'type': 'heatmap',
              'uid': '7a5dd7a5-1a5b-4498-87ee-1e862f9199fc',
              'x': array([0.        , 0.85714286, 1.71428571, 2.57142857, 3.42857143, 4.28571429,
                          5.14285714, 6.        ]),
              'y': array([ 0.        ,  0.95833333,  1.91666667,  2.875     ,  3.83333333,
                           4.79166667,  5.75      ,  6.70833333,  7.66666667,  8.625     ,
                           9.58333333, 10.54166667, 11.5       , 12.45833333, 13.